In [ ]:
import logging
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import mysql.connector

In [ ]:
logging.basicConfig(filename = 'scraper.log', level = logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def logMessage(message, level='info'):
    if level == 'info':
        logging.info(message)
    elif level == 'error':
        logging.error(message)
    else:
        logging.debug(message)

In [ ]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

response = requests.get(url)
bSoup = BeautifulSoup(response.content, 'html.parser')

tables = bSoup.find_all('table', class_='wikitable')

tableHtml = str(tables[0])
tableIo = StringIO(tableHtml)
df = pd.read_html(tableIo)[0]

logMessage('Data is successfully extracted.')
print(df.head())

In [ ]:
exchangeRates = pd.read_csv('exchange_rate.csv')

def transformData(dataframe, exchangeRates):
    dataframe['Market cap (US$ billion)'] = dataframe['Market cap (US$ billion)'].astype(str)

    dataframe['Market cap (US$ billion)'] = dataframe['Market cap (US$ billion)'].str.replace(',', '')
    
    dataframe['Market cap (US$ billion)'] = pd.to_numeric(dataframe['Market cap (US$ billion)'], errors='coerce')

    logMessage('Data cleaned and converted.')

    for index, row in exchangeRates.iterrows():
        currency = row['Currency']
        rate = row['Rate']
        
        newColumnName = f'Market cap ({currency} billion)'
        dataframe[newColumnName] = round(dataframe['Market cap (US$ billion)'].astype(float) * rate, 2)
    
    logMessage('Data transformation completed.')
    return dataframe

transformedDf = transformData(df, exchangeRates)
print(transformedDf.head())

In [ ]:
transformedDf.to_csv('largest_banks.csv', index=False)

logMessage('Data loaded to CSV file.')

In [ ]:
conn = mysql.connector.connect(user='root', passwd='Workbench98', host='localhost', database='bankdata')
print(conn)

In [ ]:
cursor = conn.cursor()

def createTable(cursor, tableName, dataframe):
    columns = dataframe.columns
    columnDefs = []
    for column in columns:
        if dataframe[column].dtype == 'object':
            columnDefs.append(f"`{column}` TEXT")
        elif dataframe[column].dtype == 'int64':
            columnDefs.append(f"`{column}` INT")
        elif dataframe[column].dtype == 'float64':
            columnDefs.append(f"`{column}` FLOAT")
        else:
            columnDefs.append(f"`{column}` TEXT")  # Default type

    columnDefsStr = ", ".join(columnDefs)
    createTableSql = f"CREATE TABLE IF NOT EXISTS `{tableName}` ({columnDefsStr})"
    cursor.execute(createTableSql)

def insertData(cursor, tableName, dataframe):
    columns = dataframe.columns
    columnsStr = ", ".join([f"`{col}`" for col in columns])
    placeholders = ", ".join(["%s"] * len(columns))
    insertSql = f"INSERT INTO `{tableName}` ({columnsStr}) VALUES ({placeholders})"

    for row in dataframe.itertuples(index=False, name=None):
        cursor.execute(insertSql, row)

tableName = 'largestbanks'

createTable(cursor, tableName, transformedDf)

insertData(cursor, tableName, transformedDf)

conn.commit()

logMessage('Data loaded to MySQL database.')

In [ ]:
def queryDatabase(query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        result = pd.DataFrame(rows, columns=columns)
        
        logMessage(f'Query executed: {query}')
        cursor.close()
        return result
        
    except Exception as e:
        logMessage(f'Query failed: {e}', level='error')
        return None

query = f'SELECT * FROM {tableName} WHERE `Market cap (US$ billion)` > 300'
largeBanks = queryDatabase(query)
print(largeBanks)

conn.close()